請使用 ImageDataGenerator 來進行 Cifar-10 資料集的訓練，並觀察不同的圖像增強方法是否會顯著影響訓練結果

In [1]:
from tensorflow import keras
import os
batch_size = 32
num_classes = 10
epochs = 5

/home/alison/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=[32,32,3]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
#model.summary()

## Training without data augmentation

In [4]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
def cifar_generator(total_size, batch_size=32):
    while True:
        for indexs in range(0, total_size, batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels
            
cifar_gen = cifar_generator(len(x_train), batch_size)

In [6]:
filepath = "./saved_models/Day99_without.h5"
if(os.path.isfile(filepath)):
    model.load_weights(filepath)
    
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
1562/1562 [==============================] - 222s 142ms/step - loss: 1.7565 - acc: 0.3522 - val_loss: 1.3304 - val_acc: 0.5162
Epoch 2/5
1562/1562 [==============================] - 222s 142ms/step - loss: 1.2852 - acc: 0.5399 - val_loss: 1.0215 - val_acc: 0.6387
Epoch 3/5
1562/1562 [==============================] - 222s 142ms/step - loss: 1.0410 - acc: 0.6301 - val_loss: 0.9060 - val_acc: 0.6825
Epoch 4/5
1562/1562 [==============================] - 222s 142ms/step - loss: 0.8961 - acc: 0.6850 - val_loss: 0.8218 - val_acc: 0.7170
Epoch 5/5
1562/1562 [==============================] - 222s 142ms/step - loss: 0.7882 - acc: 0.7251 - val_loss: 0.7746 - val_acc: 0.7348
Test loss: 0.7746459471702576
Test accuracy: 0.7348


In [ ]:
model.save_weights(filepath)

## Training with data augmentation

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_test = x_test.astype('float32')
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
def cifar_aug_generator(total_size, batch_size=32):
    while True:
        for indexs in range(0, total_size, batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            images = next(data_generator.flow(images, shuffle=False))
            images = images.astype('float32')
            images /= 255
            yield images, labels
             
cifar_gen = cifar_aug_generator(len(x_train), batch_size)

In [10]:
filepath = "./saved_models/Day99_with.h5"
if(os.path.isfile(filepath)):
    model.load_weights(filepath)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
1562/1562 [==============================] - 228s 146ms/step - loss: 1.2028 - acc: 0.5747 - val_loss: 0.9061 - val_acc: 0.6794
Epoch 2/5
1562/1562 [==============================] - 227s 145ms/step - loss: 1.1232 - acc: 0.6055 - val_loss: 0.8691 - val_acc: 0.7021
Epoch 3/5
1562/1562 [==============================] - 227s 145ms/step - loss: 1.0692 - acc: 0.6256 - val_loss: 0.7773 - val_acc: 0.7277
Epoch 4/5
1562/1562 [==============================] - 226s 145ms/step - loss: 1.0352 - acc: 0.6389 - val_loss: 0.7841 - val_acc: 0.7293
Epoch 5/5
1562/1562 [==============================] - 227s 145ms/step - loss: 1.0050 - acc: 0.6484 - val_loss: 0.7308 - val_acc: 0.7511
Test loss: 0.7308136219024658
Test accuracy: 0.7511


In [ ]:
model.save_weights(filepath)

## Training merge data

In [14]:
import numpy as np
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = np.concatenate((x_train / 255, next(data_generator.flow(x_train, batch_size=len(x_train), shuffle=False))), axis=0)
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(np.concatenate((y_train, y_train), axis=0), num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
filepath = "./saved_models/Day99_merge.h5"
if(os.path.isfile(filepath))
    model.load_weights(filepath)
    
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(0.01, 0.9),
              metrics=['accuracy'])

history = model.fit_generator(cifar_gen,
                              steps_per_epoch=len(x_train)//batch_size,
                              epochs=epochs,
                              verbose=1,
                              validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/5
3125/3125 [==============================] - 442s 142ms/step - loss: 2.3085 - acc: 0.0995 - val_loss: 1.0684 - val_acc: 0.6377
Epoch 2/5
3125/3125 [==============================] - 442s 142ms/step - loss: 2.3033 - acc: 0.0992 - val_loss: 1.0844 - val_acc: 0.6341
Epoch 3/5
3125/3125 [==============================] - 442s 142ms/step - loss: 2.3030 - acc: 0.0971 - val_loss: 1.0960 - val_acc: 0.6257
Epoch 4/5
3125/3125 [==============================] - 442s 142ms/step - loss: 2.3029 - acc: 0.0968 - val_loss: 1.3030 - val_acc: 0.5773
Epoch 5/5
3125/3125 [==============================] - 443s 142ms/step - loss: 2.3029 - acc: 0.0974 - val_loss: 1.1829 - val_acc: 0.6087
Test loss: 1.1829198835372925
Test accuracy: 0.6087


In [16]:
model.save_weights(filepath)